# MSA 2024 Phase 2 - Part 3

summary:
In this notebook, we performed the following steps:

1. Organized the image data into subfolders based on their class labels using the provided CSV file.
2. Loaded the image data from the organized directories and split it into training, validation, and test sets using data generators.
3. Built a Convolutional Neural Network (CNN) model to classify the CIFAR-10 images.
4. Trained the model on the training set and evaluated it on the validation set.
5. Predicted the classes of the test set images using the trained model and saved the predictions.

Our CNN model achieved a prediction on the test dataset. The evaluation metrics could not be calculated without the true labels. Future steps could include providing the true labels for the test set to perform a comprehensive evaluation.

### Next Steps

1. Experiment with different CNN architectures, such as deeper networks or architectures like ResNet and VGG.
2. Perform hyperparameter tuning using techniques like grid search or random search.
3. Apply data augmentation techniques to artificially increase the size of the training set.
4. Gather more data to train the model, if possible, to improve its generalization capabilities.
5. Continuously validate the model's performance on new data to ensure its robustness.

**Before start working on the competition, please ensure all required libraries are installed and properly set up on your system**:

- `python >= 3.6`,
- `tensorFlow >= 2.0`,
- `keras >= 2.3`,

and any neccassary liburaries for data manipulation and processing, e.g., `numpy`, `pandas`, etc.

In [ ]:
import tensorflow as tf
import numpy as np

### 1. Data loading & preprocessing

The CIFAR-10 dataset contains 60,000 images(32x32x3) in 10 different classes, with 6,000 images in each class. You can download the dataset directly from the competition webpage.

**To train the model, you are expected to use the training label provided in train.csv**.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

# set path
train_csv_path = 'D:/Download/nzmsa-2024/train.csv'
train_dir = 'D:/Download/nzmsa-2024/cifar10_images/train'
organized_train_dir = 'D:/Download/nzmsa-2024/cifar10_images/organized_train'
test_dir = 'D:/Download/nzmsa-2024/cifar10_images/test'
test_organized_dir = 'D:/Download/nzmsa-2024/cifar10_images/test_organized'

# load csv file
labels_df = pd.read_csv(train_csv_path)


# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Training set and validation set generator
train_generator = train_datagen.flow_from_directory(
    organized_train_dir,
    target_size=(32, 32),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    seed=101
)

validation_generator = train_datagen.flow_from_directory(
    organized_train_dir,
    target_size=(32, 32),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    seed=101
)

# Test set generator
test_generator = test_datagen.flow_from_directory(
    test_organized_dir,
    target_size=(32, 32),
    batch_size=64,
    class_mode=None,  
    shuffle=False
)

# Check if test_generator successfully loaded the image
print(f"Found {test_generator.samples} images belonging to {test_generator.num_classes} classes in test set.")

train_labels = train_generator.classes


### 2. Build & train the model

This code demostrates a simple Multi-Layer Perceptron (MLP) model. However, you are encouraged to experiment with more complex deep learning models and techniques to boost your performance.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
# Build the neural network model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 50:
        return lr
    else:
        return lr * 0.99

lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping, lr_scheduler])

# Predict on training and testing sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Print training and testing accuracy
print("Train R^2:", r2_score(y_train, y_train_pred))
print("Test R^2:", r2_score(y_test, y_test_pred))

# Evaluate the model
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("Train MSE:", mse_train)
print("Test MSE:", mse_test)
print("Train MAE:", mae_train)
print("Test MAE:", mae_test)

# Visualize actual vs predicted values
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Train set")

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Test set")

plt.tight_layout()
plt.show()

# Save test set predictions
test_filenames = [f'image_{i}.png' for i in range(len(y_test_pred))]
results = pd.DataFrame({
    'Filename': test_filenames,
    'Prediction': y_test_pred.flatten()
})
results.to_csv('D:/Download/nzmsa-2024/cifar10_images/test_predictions.csv', index=False)
